In [2]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from collections import Counter

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras import regularizers, initializers



Using TensorFlow backend.


In [3]:
def create_dictionary(texts, vocab_size):
    """
    Creates a dictionary that maps words to ids. More frequent words have lower ids.
    The dictionary contains at the vocab_size-1 most frequent words (and a placeholder '<unk>' for unknown words).
    The place holder has the id 0.
    """
    counter = Counter()
    for tokens in texts:
        counter.update(tokens)
    vocab = [w for w, c in counter.most_common(vocab_size - 1)]
    word_to_id = {w: (i + 1) for i, w in enumerate(vocab)}
    word_to_id[UNKNOWN_TOKEN] = 0
    return word_to_id


def to_ids(words, dictionary):
    """
    Takes a list of words and converts them to ids using the word2id dictionary.
    """
    ids = []
    for word in words:
        ids.append(dictionary.get(word, dictionary[UNKNOWN_TOKEN]))
    return ids

def compute_class_weights(df):
    class_weights = []
    for idx, em in enumerate(EMOTIONS):
        class_weights.append(df[em].value_counts()[1]/1000)
    print(class_weights)
    class_weights = [np.exp(i) for i in class_weights]
    sum_class_weights = sum(class_weights)
    class_weights = [ - round(i / sum_class_weights, 3) for i in class_weights]
    return {idx:class_weight for idx, class_weight in enumerate(class_weights)}

def read_data(train_file, dev_file):
    tokenizer = TweetTokenizer()
    trainDF = pd.read_csv(train_file, sep='\t')
    devDF = pd.read_csv(dev_file, sep='\t')

    allDF = pd.concat([trainDF, devDF], ignore_index=True)
    allDF = allDF.reindex(np.random.permutation(allDF.index))
    allDF.insert(1, 'tweet_tokenized', (allDF['Tweet'].apply(lambda x: tokenizer.tokenize(x))))

    word2id = create_dictionary(allDF["tweet_tokenized"], VOCAB_SIZE)

    allDF.insert(1, 'tweet_ids', (allDF['Tweet'].apply(lambda x: to_ids(x, dictionary=word2id))))

    allDF['all'] = allDF.iloc[:, -11:].values.tolist()
    
    # calc class weights
    class_weights = compute_class_weights(allDF)
    print(class_weights)
    
    total = len(allDF)
    trainend = int(total * 0.8)
    devend = trainend + int(total * 0.1)
    return allDF.iloc[:trainend, :], allDF.iloc[trainend:devend, :], allDF.iloc[devend:, :], class_weights


def evaluate(predictions, y_test):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    all_correct = 0
    for i, pred in enumerate(predictions):
        for j, em in enumerate(pred):
            if em >= 0.5:
                if y_test[i][j] == 1:
                    tp += 1
                else:
                    fp += 1
            if em <= 0.5:
                if y_test[i][j] == 1:
                    fn += 1
                else:
                    tn += 1
            if tp + tn == 11:
                all_correct += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2*(precision * recall) / (precision + recall)

    print("F1: {}\nPrecision: {}\nRecall: {}\nCompletely correct: {}".format(f1, precision, recall, all_correct))


In [4]:
data_dir = 'D:/3_Programming/1_Studium/Python/SemEval2018_Task1_5/data/'
train_file = os.path.join(data_dir, '2018-E-c-En-train.txt')
dev_file = os.path.join(data_dir, '2018-E-c-En-dev.txt')

VOCAB_SIZE = 100000
MAX_LEN = 100
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 50
EPOCHS = 10  # Standard 10
UNKNOWN_TOKEN = "<unk>"
EMOTIONS = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
            'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

In [5]:
trainDF, devDF, testDF, class_weights = read_data(train_file, dev_file)
x_train = sequence.pad_sequences(np.array(trainDF['tweet_ids']), maxlen=MAX_LEN)
x_dev = sequence.pad_sequences(np.array(devDF['tweet_ids']), maxlen=MAX_LEN)
x_test = sequence.pad_sequences(np.array(testDF['tweet_ids']), maxlen=MAX_LEN)
y_train = np.array([trainDF['all']])[0]
y_dev = np.array([devDF['all']])[0]
y_test = np.array([testDF['all']])[0]


[2.859, 1.1020000000000001, 2.9209999999999998, 1.363, 2.8769999999999998, 0.83199999999999996, 2.2909999999999999, 0.89500000000000002, 2.2730000000000001, 0.39600000000000002, 0.40000000000000002]
{0: -0.19800000000000001, 1: -0.034000000000000002, 2: -0.21099999999999999, 3: -0.043999999999999997, 4: -0.20200000000000001, 5: -0.025999999999999999, 6: -0.112, 7: -0.028000000000000001, 8: -0.11, 9: -0.017000000000000001, 10: -0.017000000000000001}


In [6]:
cnn_model = Sequential()
cnn_model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
cnn_model.add(Conv1D(2 * HIDDEN_SIZE,
                     kernel_size=3,
                     activation='tanh',
                     strides=1,
                     padding='valid',
                     ))
# cnn_model.add(GlobalMaxPooling1D())
# cnn_model.add(Dropout(0.5))
# cnn_model.add(Conv1D(2 * HIDDEN_SIZE,
#                      kernel_size=3,
#                      activation='tanh',
#                      strides=1,
#                      padding='valid',
#                      ))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(HIDDEN_SIZE, activation='tanh'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(11, activation='sigmoid')) #11 = no of classes

In [7]:
from keras.optimizers import SGD, adam

opt = adam(lr=0.01)
cnn_model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])


In [8]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopper = EarlyStopping(monitor='val_acc', patience=5, mode='max')
checkpoint = ModelCheckpoint(data_dir+'model.m', save_best_only=True, monitor='val_acc', mode='max')


#Todo:Functional API class:weights 0,1 11 outputs
"It depends on your application. Class weights are useful when training on highly skewed data sets; for example, a classifier to detect fraudulent transactions. Sample weights are useful when you don't have equal confidence in the samples in your batch. A common example is performing regression on measurements with variable uncertainty." https://stackoverflow.com/questions/43459317/keras-class-weight-vs-sample-weights-in-the-fit-generator

In [9]:
cnn_model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopper, checkpoint],
    epochs=EPOCHS,
    validation_data=(x_dev, y_dev),
    class_weight=class_weights,
    verbose=1
)


Train on 6179 samples, validate on 772 samples
Epoch 1/10


  32/6179 [..............................] - ETA: 2:46 - loss: -0.0990 - acc: 0.5455

  64/6179 [..............................] - ETA: 1:38 - loss: -0.1148 - acc: 0.4588

  96/6179 [..............................] - ETA: 1:15 - loss: -0.1320 - acc: 0.4025

 128/6179 [..............................] - ETA: 1:03 - loss: -0.1503 - acc: 0.3729

...

3712/6179 [=================>............] - ETA: 12s - loss: -1.9569 - acc: 0.2365

3744/6179 [=================>............] - ETA: 12s - loss: -1.9576 - acc: 0.2365

3776/6179 [=================>............] - ETA: 12s - loss: -1.9586 - acc: 0.2365

3808/6179 [=================>............] - ETA: 11s - loss: -1.9584 - acc: 0.2363

KeyboardInterrupt: 

In [92]:
from keras.models import load_model
best_model = load_model(data_dir+'model.m')

In [93]:
predictions = best_model.predict(x_test)
print(y_test[0])
print(predictions[0])

[0 0 0 0 0 0 0 1 1 0 0]
[ 0.2445745   0.1203005   0.26531667  0.10367718  0.55123574  0.16632465
  0.49936083  0.10544037  0.20550883  0.07489073  0.07487898]


In [94]:
evaluate(predictions,y_test)

F1: 0.13215859030837004
Precision: 0.5844155844155844
Recall: 0.07450331125827815
Completely correct: 1


Unweighted 
	F1: 0.27632915678970543
	Precision: 0.3633125556544969
	Recall: 0.22295081967213115
	Completely correct: 2

In [ ]:
Softmax
F1: 0.13215859030837004
Precision: 0.5844155844155844
Recall: 0.07450331125827815
Completely correct: 1